# CRS pitfalls

When working with `osdatahub` you will notice that some of the methods on `Extent` have an argument called CRS. For example when creating a bounding box you need to specify a CRS like this `extent = Extent.from_bbox((-1, -1, 1, 1), "EPSG:4326")`. The second argument is called CRS, which stand for "coordinate reference system". A CRS is needed when processing geo data in order to manipulate the coordinates, typically you won't need to know much about the CRS when using `osdatahub` but there are some common "gotchas" which we will describe here.

A CRS may sometimes be referred to as a SRS (spatial reference system) but they are the same thing.

In `osdatahub` the CRS is specified as a string in the following format "EPSG:27700". EPSG stands for European Petroleum Survey Group and 27700 represents the CRS which Ordnance Survey uses.

# Consequences of having multiple CRS

### How can the same coordinates can represent different locations
Consider the image below, it shows the same region but each train station (Totton and Redbridge) have decided to use their own coordinate system. This is analaogus to what happens in the real world, for example Ordnance Survey and GPS use a different CRS.

If you were to use the Totton CRS, then you would find that the coordinates of Redbridge are approximately `(x, y)=(8.5, 2.9)`. However, the location of Redbridge using the Redbridge CRS is `(x, y)=(0, 0)`. Therefore, if you wanted to tell someone where Redbridge was, and said it is at (8.5, 2.9) but didn't tell them what CRS the coordinates are in, then they can have no way of knowing what location you actually mean.

![CRS](https://raw.githubusercontent.com/OrdnanceSurvey/osdatahub/master/Examples/images/CRS.png "CRS")

Note that CRS doesn't just tell you where to put your for your coordinate axes but also contains information on how much a a unit is. In the Redbridge CRS note that the spaces between axis lines is much larger than in the Totton CRS. There are also many other parameters that go into a CRS, things to describe the shape of the earth, how its rotated and many others.

A full description of CRS is not needed to use `osdatahub` but if you are interested in that kind of thing this link is useful for grounding your understanding https://datacarpentry.org/organization-geospatial/03-crs/

# Specifing the wrong bounding box

The most likely pitfall, due to CRS, you may encouter relates to specifying your extent when creating an `Extent` for `osdatahub`.

Recall that you can make an `Extent` object from a bounding box like so.

This bounding box is for a portion of Westminster

In [ ]:
from osdatahub import Extent
extent = Extent.from_bbox((530034, 180154, 530699, 180381), "EPSG:27700")

But a very common mistake is to specify the coordiates in a different CRS to the CRS passed to the method. Suppose we made the following error.

In [ ]:
extent = Extent.from_bbox((530034, 180154, 530699, 180381), "EPSG:4326") # incorrect CRS argument

Here we have given the coordinates in the CRS 27700, however we've mistakenly specified the CRS as 4326. In the Redbridge/Totton example we saw that the same coordinates in different CRS can represent different locations. For this example the bounding box is actually somewhere in the arctic circle, and consequently any query for OS data in this region would return an empty list.

##### Sanity checks
A good way to check if you are using the correct CRS for your coordinates is to ensure the coordinates fall within a sensible range.

- "EPSG:27700" This CRS is called British Nation Grid (BNG) it is used by Ordnance Survey to map the British Isles. In this system x coordinates are in the range -90619.29 to 612435.55 and y is in the range 10097.13 to 1234954.16
- "EPSG:4326" called World Geodetic System 1984 (WGS84) is used by GPS and has the following ranges of values -180 to 180.0 and -90.0 to 90.0
- "EPSG:3857" Pseudo-Mercator is used by mapping companies like google and open street map with values between -180 to 180.0 and -85.6 to 85.6

This means its pretty easy to see if you are using BNG when you should be WGS84 or Pseudo-Mercator. Differentiating between WGS84 and Pseudo-Mercator can't be done with this check and will require you to check which CRS you want.